In [3]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys
import tensorflow_addons as tfa


In [4]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

# model = SwinTransformer3D(**cfg)

In [5]:
def get_model(num_classes,cfg, shape_of_input=(10,3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)
    x = backbone(inputs, training= False)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [6]:
shape_of_input = (5, 3, 8, 112,112)
model = get_model(5, cfg, shape_of_input=shape_of_input)

X = tf.random.normal(shape_of_input)
y = tf.random.uniform((shape_of_input[0],1), 0, 5, tf.dtypes.int32)
model.summary()

shape_of_input:  (5, 3, 8, 112, 112)
5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 8, 112, 112)]  0         
                                                                 
 swin_transformer3d (SwinTra  (None, 768, 4, 4, 4)     29694438  
 nsformer3D)                                                     
                                                                 
 i3d_head_tf (I3DHead_tf)    (None, 5)                 3845      
                                                                 
Total params: 29,698,283
Trainable params: 27,854,315
Non-trainable params: 1,843,968
_________________________________________________________________


In [ ]:
# inputs = tf.keras.Input(shape_of_input[1:])
# model(inputs, training=True)

In [7]:
y1 = model.predict(X)
y1

1/1 [==============================] - 7s 7s/step


array([[0.21804325, 0.17564876, 0.20801349, 0.21439712, 0.1838974 ],
       [0.2160434 , 0.16880102, 0.22613603, 0.20458852, 0.18443106],
       [0.21588865, 0.16063978, 0.24322826, 0.20265196, 0.17759134],
       [0.2376091 , 0.14616466, 0.22391437, 0.22256124, 0.16975066],
       [0.2341917 , 0.15283717, 0.22144271, 0.20615363, 0.18537486]],
      dtype=float32)

In [2]:
optimizer = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=3e-4, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)

## pariwise constant decay

In [14]:


metrics=["top_k_categorical_accuracy", "categorical_accuracy"] 
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=optimizer, metrics=metrics)

model.fit(X, y, epochs=2)

Epoch 1/2
1/1 [==============================] - 21s 21s/step - loss: 1.0030 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8000
Epoch 2/2
1/1 [==============================] - 5s 5s/step - loss: 0.6247 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.4000


In [ ]:
y1 = model.predict(X)
y1

In [ ]:
y2 = model(X, training=False)
y2

In [ ]:
model.layers[1].layers[5].weights